In [ ]:
import os

import numpy as np
import pandas as pd
import random

from agents.matchmanager import buildMatchManager

from multiprocessing import Pool

In [ ]:
agents = ['RandomAgent','AlphaBetaAgent','GreedyAgent']
scenarios = ['scenarioTest1v1', 'scenarioTest2v2', 'scenarioJunctionExo', 'scenarioJunction', 'scenarioRoadblock', 'scenarioBridgeHead', 'scenarioCrossingTheCity']

In [ ]:
def compress(scenario):
    
    files = [f for f in os.listdir('data') if scenario in f]
    
    dfs = [pd.concat([pd.read_csv(os.path.join('data', f), index_col=0) for f in files[i:i+1000]]) for i in range(0, len(files), 1000)]
    
    dfs = pd.concat(dfs)
    
    dfs.to_pickle(f'data.{scenario}.pkl.gz', compression='gzip')

In [ ]:
with Pool(len(scenarios)) as p:
    p.map(compress, scenarios)

In [ ]:
def game(args):
    scenario, red, blue, seed = args
    vectors=[]

    mm = buildMatchManager('', scenario, red, blue, seed=seed)

    while not mm.end:
        mm.nextStep()
        v = list(mm.state.vector())
        vectors.append(v)
    
    for v in vectors:
        v += [mm.winner, scenario, red, blue, seed]
    
    cols = list(mm.state.vectorInfo()) + ['winner', 'meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed']
        
    df = pd.DataFrame(data=vectors, columns=cols)
    df.to_csv(f'data/{scenario}.{red}.{blue}.{seed}.csv')


In [ ]:
game(('scenarioTest1v1', 'GreedyAgent', 'GreedyAgent', 1))

In [ ]:
args = []

np.random.seed(42)

for _ in range(1000000):
    scenario = np.random.choice(scenarios)
    red = np.random.choice(agents)
    blue = np.random.choice(agents)
    seed = np.random.randint(100000000, 999999999)
    
    args.append((scenario, red, blue, seed))

In [ ]:
with Pool(64) as p:
    p.map(game, args)